In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/capstone24-35/capstone-2024-35/Data Extract/colab

/content/drive/MyDrive/capstone24-35/capstone-2024-35/Data Extract/colab


In [ ]:
!pip install torch transformers datasets
!pip install git+https://github.com/SKTBrain/KoBERT.git@master
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install kobert-transformers
!pip install transformers
!pip install torch
!pip install datasets
!pip install seqeval  # NER 평가를 위한 라이브러리
!pip install transformers[torch] -U

from transformers import BertTokenizerFast, BertForTokenClassification, AdamW
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from kobert_tokenizer import KoBERTTokenizer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
dataset = load_dataset("klue", "ner")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/21008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:

from kobert_transformers import get_kobert_model, get_tokenizer
from transformers import BertForTokenClassification

# KoBERT 모델과 토크나이저 로드
model = get_kobert_model()
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

# 토큰 분류를 위한 KoBERT 모델 초기화
model = BertForTokenClassification.from_pretrained('skt/kobert-base-v1', num_labels=13)  # NER에 사용할 라벨 수 설정


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_and_align_labels(batch):
    tokenized_inputs = tokenizer(batch['tokens'], is_split_into_words=True, truncation=True, padding='max_length', max_length=512)
    labels = []

    for i, (tokens, doc_labels) in enumerate(zip(batch['tokens'], batch['ner_tags'])):
        # 토크나이저가 처리한 입력에 대한 'input_ids'
        input_ids = tokenized_inputs['input_ids'][i]

        # 토큰화된 결과와 원본 라벨을 매핑
        aligned_labels = []
        label_idx = 0

        for token_id in input_ids:
            # 토크나이저로부터 토큰을 얻음
            token = tokenizer.decode([token_id])

            # '[CLS]', '[SEP]', 패딩 등의 특수 토큰 처리
            if token_id in (tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id):
                aligned_labels.append(-100)  # 학습에서 무시
                continue

            # 서브워드가 아니거나 새로운 단어의 시작일 때
            if token.startswith("▁") or label_idx == 0:
                aligned_labels.append(doc_labels[label_idx])
                if label_idx < len(doc_labels) - 1:
                    label_idx += 1
            else:
                aligned_labels.append(-100)  # 학습에서 무시될 토큰에 대해 특별한 라벨 지정

        labels.append(aligned_labels)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
from datasets import DatasetDict


# 데이터셋을 무작위로 섞는다. 여기서 dataset['train']을 사용하여 훈련 데이터셋에 접근.
shuffled_train_dataset = dataset['train'].shuffle(seed=42)

# 훈련 데이터셋에서  샘플을 선택.
selected_train_dataset = shuffled_train_dataset.select(range(10000))



# 선택된 데이터셋을 다시 섞어 검증셋을 분리.
shuffled_selected_train_dataset = selected_train_dataset.shuffle(seed=42)
valid_dataset_size = int(len(selected_train_dataset) * 0.1)  # 10%를 검증 데이터셋으로 사용.

# 검증 데이터셋 분리
valid_dataset = shuffled_selected_train_dataset.select(range(valid_dataset_size))

# 트레인 데이터셋 분리 (검증 데이터셋을 제외한 나머지)
train_dataset = shuffled_selected_train_dataset.select(range(valid_dataset_size, len(selected_train_dataset)))

# 새로운 데이터셋 구조를 생성.
split_dataset = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset
})

# 전처리 함수 적용.
tokenized_datasets = split_dataset.map(tokenize_and_align_labels, batched=True, remove_columns=split_dataset["train"].column_names)


Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [32]:
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=8)
eval_loader = DataLoader(eval_dataset, batch_size=8)


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/capstone24-35/capstone-2024-35/Data Extract/model_finetuned",                  # 모델 출력 디렉토리
    num_train_epochs=3,                      # 훈련할 총 에포크 수
    per_device_train_batch_size=4,           # 각 디바이스별 훈련 배치 크기
    per_device_eval_batch_size=4,            # 각 디바이스별 평가 배치 크기
    warmup_steps=500,                        # 웜업 스텝 수
    weight_decay=0.01,                       # 가중치 감소
    evaluation_strategy='epoch',             # 에포크마다 평가
    logging_dir="./logs",                    # 로깅 디렉토리
    save_strategy="epoch",                   # 에포크마다 모델 저장
    load_best_model_at_end=True,             # 훈련이 끝날 때 최고의 모델을 로드
    save_total_limit=3                       # 최대 저장할 모델 파일 수, 오래된 것부터 삭제
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)



/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()
trainer.evaluate()


Epoch,Training Loss,Validation Loss
1,1.355400,0.579345


Epoch,Training Loss,Validation Loss
1,1.355400,0.579345
2,0.731500,0.467709
3,0.315600,0.527938


In [29]:
from datasets import load_dataset

# KLUE NER 데이터셋 로드
dataset = load_dataset("klue", "ner")

# 태그 리스트 확인
tag_list = dataset['train'].features['ner_tags'].feature.names
print(tag_list)

# tag2id 및 id2tag 사전 생성
tag2id = {tag: id for id, tag in enumerate(tag_list)}
id2tag = {id: tag for tag, id in tag2id.items()}


['B-DT', 'I-DT', 'B-LC', 'I-LC', 'B-OG', 'I-OG', 'B-PS', 'I-PS', 'B-QT', 'I-QT', 'B-TI', 'I-TI', 'O']


In [30]:
from transformers import BertForTokenClassification, BertTokenizerFast, Trainer, TrainingArguments
from kobert_transformers import get_kobert_model, get_tokenizer

checkpoint_path = "/content/drive/MyDrive/capstone24-35/capstone-2024-35/Data Extract/model/checkpoint-2250"
model = BertForTokenClassification.from_pretrained(checkpoint_path, num_labels=13)  # num_labels는 데이터셋의 라벨 수에 맞춰 조정
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [33]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/capstone24-35/capstone-2024-35/Data Extract/model_finetuned",  # 추가 학습 후 모델 저장 경로
    num_train_epochs=3,  # 추가 훈련할 에포크 수
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    logging_dir="./logs_retrained",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=3
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.325000,0.439925
